In [ ]:
pip install pandas
pip install numpy
pip install seaborn
pip install bertopic

In [2]:
import re
import pandas as pd
import json
import numpy as np
import seaborn as sns
import nltk
from bertopic import BERTopic

/home/rodrigo98rm/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-24 19:35:53.118017: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-24 19:35:53.118032: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [55]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df

tweets = df_tweets_candidatos('./datasets/dataset.json')


# Adicionar nomes dos candidatos no dataframe
for index, row in tweets.iterrows():
    candidato = ''
    match row.author_id:
        case 2670726740:
            candidato = 'lula'
        case 128372940:
            candidato = 'bolsonaro'
        case 33374761:
            candidato = 'ciro'
        case _:
            candidato: 'n/d'
    tweets.at[index, 'candidato'] = candidato

# Embaralhar tweets
tweets = tweets.sample(frac=1)
        

print(f'Número de tweets: {len(tweets)}\n')
print(tweets.head())

Número de tweets: 300

      author_id      conversation_id                created_at  \
43   2670726740  1559593295956873216 2022-08-16 17:29:52+00:00   
46   2670726740  1559507556694990848 2022-08-16 12:38:27+00:00   
270   128372940  1556999066235670528 2022-08-09 13:41:20+00:00   
7    2670726740  1560226220368969728 2022-08-18 11:24:53+00:00   
3    2670726740  1560266077329694720 2022-08-18 14:03:15+00:00   

                                                  text                   id  \
43   Lula dá início a campanha em porta de fábrica ...  1559593295956873216   
46   @foquinha Obrigado pelo apoio, @foquinha. Nós ...  1559519957897076736   
270  - Esses trilhos são um importante avanço para ...  1556999069263855616   
7    Videogame não é apenas entretenimento, é també...  1560226220368969728   
3    RT @verdadenarede: Damares condenada! No final...  1560266077329694720   

     candidato  
43        lula  
46        lula  
270  bolsonaro  
7         lula  
3         lula  


Normalizar dados e remover tokens com pouco significado

In [3]:
nltk.download('stopwords')
pt_stop = set(nltk.corpus.stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rodrigo98rm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):

        # removing urls
        document = re.sub(r'http\S+', '', document)

        # removing mentions
        document = re.sub(r'\B@\w+', '', document)

        # remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # converting to lowercase
        document = document.lower()

        # lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in pt_stop]
        tokens = [word for word in tokens if len(word) > 3]

        return ' '.join(tokens)

In [58]:
# create model 
model = BERTopic(language="multilingual", verbose=True, min_topic_size=3, top_n_words=10, calculate_probabilities=True)

# convert to list 
docs = tweets.text.to_list()

# Pre-processamento
for i, doc in enumerate(docs):
    docs[i] = preprocess_text(doc)

tweets_topic, probabilities = model.fit_transform(docs)

Lula dá início a campanha em porta de fábrica no ABC https://t.co/xrEMsi6Jfs
lula início campanha porta fábrica


Batches: 100%|██████████| 10/10 [00:00<00:00, 49.22it/s]
2022-08-24 20:03:03,078 - BERTopic - Transformed documents to Embeddings
2022-08-24 20:03:04,566 - BERTopic - Reduced dimensionality
2022-08-24 20:03:04,595 - BERTopic - Clustered reduced embeddings


Adicionar no dataframe o tópico identificado para cada tweet

In [59]:

topic_names = model.generate_topic_labels(nr_words=4)

# Array indicando o tópico de cada tweet (numérico -> índice do array "topic_names" + 1)
# print(tweets_topic)
# Array com todos os tópicos nomeados
# print(topic_names)

for index, tweet_topic in enumerate(tweets_topic):
    tweets.at[index + 1, 'topico'] = topic_names[tweet_topic + 1]

tweets.head()


,author_id,conversation_id,created_at,text,id,candidato,topico
43,2.670727e+09,1.559593e+18,2022-08-16 17:29:52+00:00,Lula dá início a campanha em porta de fábrica ...,1.559593e+18,lula,1_melhor_brasildaesperança_esperança_brasil
46,2.670727e+09,1.559508e+18,2022-08-16 12:38:27+00:00,"@foquinha Obrigado pelo apoio, @foquinha. Nós ...",1.559520e+18,lula,12_garantido_programa_renda_constitucional
270,1.283729e+08,1.556999e+18,2022-08-09 13:41:20+00:00,- Esses trilhos são um importante avanço para ...,1.556999e+18,bolsonaro,-1_ciro_brasil_democracia_hoje
7,2.670727e+09,1.560226e+18,2022-08-18 11:24:53+00:00,"Videogame não é apenas entretenimento, é també...",1.560226e+18,lula,24_rota_ciropresidente12_acompanha_democráticos
3,2.670727e+09,1.560266e+18,2022-08-18 14:03:15+00:00,RT @verdadenarede: Damares condenada! No final...,1.560266e+18,lula,-1_ciro_brasil_democracia_hoje


Abaixo, os tópicos identificados entre todos os tweets

In [60]:
freq = model.get_topic_info()
print(freq)

    Topic  Count                                             Name
0      -1     63                   -1_ciro_brasil_democracia_hoje
1       0     25                     0_luladay_noite_gettr_baixar
2       1     24      1_melhor_brasildaesperança_esperança_brasil
3       2     16                  2_pequenos_deixar_custou_quanto
4       3     15                   3_famílias_milhões_safra_renda
5       4     14             4_redução_energia_combustíveis_preço
6       5     13                     5_deus_cristãos_crime_adorar
7       6     11                         6_brasil_fome_vamos_país
8       7     11                        7_eleição_voto_falar_café
9       8      9        8_veio_repare_cirocomandrémarinho_relação
10      9      9                          9_2022_giro_monte_claro
11     10      8                    10_torcida_todos_vivo_jéssica
12     11      7              11_programa_renda_mínima_distribuir
13     12      6       12_garantido_programa_renda_constitucional
14     13 

Visualização de tweets com tópicos semelhantes:

In [6]:
model.visualize_documents(docs)

In [8]:
model.visualize_topics()

In [7]:
model.visualize_barchart(top_n_topics=5)

In [9]:
model.visualize_heatmap(width=750, height=750)

In [10]:
model.visualize_term_rank()

TODO:
- Normalize, remove stop words, lower case, etc